In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.1110, -0.0908, -0.0593,  ..., -0.0210,  0.0073,  0.1354],
        [ 0.0803, -0.1146, -0.0505,  ...,  0.0389,  0.0015, -0.0009],
        [ 0.1095, -0.1104,  0.1180,  ..., -0.0240,  0.0285,  0.2381],
        ...,
        [ 0.0846, -0.0130, -0.0164,  ..., -0.0324, -0.0259,  0.1343],
        [ 0.0744, -0.0979, -0.0547,  ...,  0.0056, -0.0765,  0.2076],
        [ 0.1872, -0.0603,  0.0691,  ..., -0.1695,  0.0267,  0.1759]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 150
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-67.3198, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.4134, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1453, device='cuda:0', grad_fn=<NegBackward0>), tensor(-49.7610, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-38.2245, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.1243, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.8175, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.2826, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-38.2245, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.1243, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.8175, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.2826, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.0317, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.7782, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.2377, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.0317, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-125.0400, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9535, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.1672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.9193, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-125.0399, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9535, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.1672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.9192, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-126.7721, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.0712, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.1905, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.5105, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-126.7722, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.0712, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.1905, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.5105, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-130.3826, device='cuda:0', grad_fn=<NegBackw

tensor(-145.7456, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.6539, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.8095, device='cuda:0', grad_fn=<NegBackward0>), tensor(-112.2822, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-147.3959, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.7994, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.9812, device='cuda:0', grad_fn=<NegBackward0>), tensor(-113.6154, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-147.3959, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.7994, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.9812, device='cuda:0', grad_fn=<NegBackward0>), tensor(-113.6154, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-149.0010, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-20.2133, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.1022, device='cuda:0', grad_fn=<NegBackward0>), tensor(-114.6855, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-149.0011, device='cuda:0', grad_fn=<NegB

tensor(-160.0253, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.6186, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.7251, device='cuda:0', grad_fn=<NegBackward0>), tensor(-123.6816, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-160.0253, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.6186, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.7251, device='cuda:0', grad_fn=<NegBackward0>), tensor(-123.6816, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-161.4861, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.7985, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.7766, device='cuda:0', grad_fn=<NegBackward0>), tensor(-124.9110, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-161.4861, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.7985, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.7766, device='cuda:0', grad_fn=<NegBackward0>), tensor(-124.9110, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-162.6825, device='cuda:0', grad_fn=<NegB

tensor(-166.3596, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.5477, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.2676, device='cuda:0', grad_fn=<NegBackward0>), tensor(-128.5443, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-167.7257, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.6779, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.3047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-129.7431, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-167.7257, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.6779, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.3047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-129.7431, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-169.5226, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.7988, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.3342, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.3896, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-169.5226, device='cuda:0', grad_fn=<NegB

tensor(-166.4330, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.8528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.2029, device='cuda:0', grad_fn=<NegBackward0>), tensor(-128.3773, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.4330, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.8528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.2029, device='cuda:0', grad_fn=<NegBackward0>), tensor(-128.3773, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-170.0733, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.2798, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.5441, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.2494, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-170.0732, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.2798, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.5441, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.2493, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-172.3698, device='cuda:0', grad_fn=<NegB

tensor(-161.4853, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.6505, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.6734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-124.1614, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-161.9809, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.6135, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.6830, device='cuda:0', grad_fn=<NegBackward0>), tensor(-124.6844, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-161.9809, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.6135, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.6830, device='cuda:0', grad_fn=<NegBackward0>), tensor(-124.6844, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-163.8743, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.0226, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.9264, device='cuda:0', grad_fn=<NegBackward0>), tensor(-125.9253, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-163.8743, device='cuda:0', grad_fn=<NegB

tensor(-175.6741, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.5588, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.3802, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.7350, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-175.6741, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.5588, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.3802, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.7350, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.7865, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.2729, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.9406, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.5730, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.7865, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.2729, device='cuda:0', grad_fn=<NegBackward0>), tensor(-15.9406, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.5730, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-164.9554, device='cuda:0', grad_fn=<NegB

tensor(-184.6292, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.7798, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.7848, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.0646, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-184.8430, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.7355, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.7842, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.3233, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-184.8430, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.7355, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.7842, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.3233, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-185.7958, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.8886, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.9920, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.9151, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-185.7958, device='cuda:0', grad_fn=<NegB

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]


----------------------------
Train_z, Test_z
PCA: 0.8072
SVD: 0.7977
NetD, Acc at first 0.8609
